In [ ]:
import random
from numpy import exp as exp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
learning_rate=0.000001
epochs=100
batch_size=100

In [ ]:
def transpose(matrix):
  return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]
def shape(matrix):
  return len(matrix), len(matrix[0])

In [ ]:
#runtime:13-20s
with open('/content/drive/MyDrive/Colab Notebooks/MNIST_train.txt', 'r') as file:
  lines = [line.strip().split(',') for line in file]
y_train = [[0] * 60000 for _ in range(10)]
for i, row in enumerate(lines):
    label = int(row[0])
    y_train[label][i] = 1
X_train = [[float(num) for num in row[1:]] for row in lines]

In [ ]:
#runtime:4-2s
with open('/content/drive/MyDrive/Colab Notebooks/MNIST_test.txt', 'r') as file:
  lines = [line.strip().split(',') for line in file]
y_test = [[0] * 10000 for _ in range(10)]
for i, row in enumerate(lines):
    label = int(row[0])
    y_test[label][i] = 1
X_test = [[float(num) for num in row[1:]] for row in lines]

In [ ]:
#runtime:26s
X_train, X_test = map(transpose, [X_train, X_test])

In [ ]:
print(f"X_train dims: {shape(X_train)}")
print(f"X_test dims: {shape(X_test)}")
print(f"y_train dims: {shape(y_train)}")
print(f"y_test dims: {shape(y_test)}")

X_train dims: (784, 60000)
X_test dims: (784, 10000)
y_train dims: (10, 60000)
y_test dims: (10, 10000)


In [ ]:
class logisticRegression:

  def __init__(self, X):
    self.theta = [[random.uniform(-1, 1) for _ in range(10)] for _ in range(len(X))]
  def transpose(self,mat):
    return [[mat[i][j] for i in range(len(mat))] for j in range(len(mat[0]))]

  def matmul(self, A, B):
    rows_A = len(A)
    cols_A = len(A[0]) if A else 0
    rows_B = len(B)
    cols_B = len(B[0]) if B else 0
    if cols_A != rows_B:
      raise ValueError(f"Incompatible dimensions for matrix multiplication: A({rows_A}, {cols_A}) and B({rows_B}, {cols_B})")
    return [[sum(A[i][k] * B[k][j] for k in range(len(A[0]))) for j in range(len(B[0]))] for i in range(len(A))]

  def softmax(self, x):
      max_x = max(x)
      exp_x = [exp(i - max_x) for i in x]
      sum_exp_x = sum(exp_x)
      return [float(i / sum_exp_x) for i in exp_x]

  def elwise(self, A, B, operator):
    if operator == "m+m":
      C = []
      for i in range(len(A)):
        row = []
        for j in range(len(A[0])):
          row.append(A[i][j] + B[i][j])
        C.append(row)
    elif operator == "sum":
      C = 0
      for i in range(len(A)):
        row = 0
        for j in range(len(A[0])):
          row +=A[i][j]
        C+=row
    elif operator == "s*m":
      C = []
      for i in range(len(B)):
        row = []
        for j in range(len(B[0])):
          row.append(A*B[i][j])
        C.append(row)
    return C

  def batch(self, X, y, n):
    if n > len(X[0]):
      n = len(X[0])
    selected_indexes = random.sample(range(len(X[0])), n)
    X_batch = [[X[row][col] for col in selected_indexes] for row in range(len(X))]
    y_batch = [[y[row][col] for col in selected_indexes] for row in range(len(y))]
    return X_batch, y_batch

  def hypothesis(self, X):
    result = self.matmul(self.transpose(self.theta),X)
    for j in range(len(result[0])):
        column = [result[i][j] for i in range(len(result))]
        softmax_column = self.softmax(column)
        for i in range(len(result)):
            result[i][j] = softmax_column[i]
    return result

  def predict(self, X):
    sm = self.hypothesis(X)
    predictions = []
    for i in range(len(sm[0])):
        max_index = 0
        max_value = sm[0][i]
        for j in range(1, len(sm)):
            if sm[j][i] > max_value:
                max_value = sm[j][i]
                max_index = j
        predictions.append(max_index)
    return predictions

  def accuracy(self, X, y):
    predictions = self.predict(X)
    correct=0
    for i in range(len(predictions)):
      if y[predictions[i]][i] == 1:
        correct+=1
    accuracy = correct / len(predictions)
    return correct,accuracy

  def step(self, alpha, X, y):
    X_batch, y_batch = self.batch(X, y, batch_size)
    predictions = self.elwise(-1,self.hypothesis(X_batch),"s*m")
    error = self.elwise(y_batch, predictions,"m+m")
    gradient = self.matmul(X_batch, self.transpose(error))
    self.theta = self.elwise(self.theta, self.elwise(alpha, gradient, "s*m"),"m+m")

  def train(self, X, y, lr, epochs):
    global X_train, y_train, X_test, y_test
    X_train_batch, y_train_batch = self.batch(X_train, y_train, 1000)
    X_test_batch, y_test_batch = self.batch(X_test, y_test, 1000)
    print("accuracy of training data:",self.accuracy(X_train_batch, y_train_batch))
    print("accuracy of test data:",self.accuracy(X_test_batch, y_test))
    for epoch in range(epochs):
      self.step(lr, X, y)
      if epoch % 50 == 0:
        print('epoch:',epoch)
    X_train_batch, y_train_batch = self.batch(X_train, y_train, 10000)
    X_test_batch, y_test_batch = self.batch(X_test, y_test, 10000)
    print("accuracy of training data:",self.accuracy(X_train_batch, y_train_batch))
    print("accuracy of test data:",self.accuracy(X_test_batch, y_test_batch))
logReg = logisticRegression(X_train)

In [ ]:
logReg.train(X_train, y_train, learning_rate, epochs) #1 hour for 10k epochs and 100 batch size

accuracy of training data: (90, 0.09)
accuracy of test data: (84, 0.084)
epoch: 0
epoch: 50
accuracy of training data: (2129, 0.2129)
accuracy of test data: (2150, 0.215)
